<a href="https://colab.research.google.com/github/rohitpawar4507/Movie-Recomendation-System-Using-K-Means-Clustering/blob/main/Movie_Recomendation_System_Using_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Author - Rohit Pawar

# Movie Recommdation System Using K-Means Clustering

In [1]:
!pip install factor_analyzer

     |████████████████████████████████| 41 kB 416 kB/s 
  Created wheel for factor-analyzer: filename=factor_analyzer-0.4.0-py3-none-any.whl size=41455 sha256=3b29dd406cadbeaa98611adaffa38cd3a4a09231b6eb6c58404c3067e3b05934
  Stored in directory: /root/.cache/pip/wheels/ac/00/37/1f0e8a5039f9e9f207c4405bbce0796f07701eb377bfc6cc76
Successfully built factor-analyzer


In [2]:
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [3]:
from google.colab import files
uploaded = files.upload()

Saving movies data.csv to movies data.csv


In [5]:
from google.colab import files
uploaded = files.upload()

Saving ratings.csv to ratings.csv


In [50]:
from google.colab import files
uploaded = files.upload()

Saving ratings data.csv to ratings data (1).csv


In [58]:
#Importing both the file using pandas 
data1 = pd.read_csv('movies data.csv')
data2 = pd.read_csv('ratings data.csv')

In [59]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3883 non-null   int64 
 1   MovieID     3883 non-null   int64 
 2   Title       3883 non-null   object
 3   Genres      3883 non-null   object
dtypes: int64(2), object(2)
memory usage: 121.5+ KB


In [60]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394164 entries, 0 to 394163
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Unnamed: 0  394164 non-null  int64
 1   UserID      394164 non-null  int64
 2   MovieID     394164 non-null  int64
 3   Rating      394164 non-null  int64
 4   Timestamp   394164 non-null  int64
dtypes: int64(5)
memory usage: 15.0 MB


In [61]:
#Deleting unnecessary columns
data1 = data1.drop('Unnamed: 0',axis = 1)
data2 = data2.drop(['Unnamed: 0','Timestamp'],axis = 1)

In [62]:
data1.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [63]:
data2.head()

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [64]:
#Merging both the dataframes
data = pd.merge(data2 , data1 , how='outer', on='MovieID')

In [65]:
data.head(5)

,UserID,MovieID,Rating,Title,Genres
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama


In [66]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = data.Genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,Children's,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['Genres'].str.contains(i), i] = 1

In [68]:
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,Children's,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
# Now there is no use of genre 
# Since we have movie id so there is no need for movie names as well
data = data.drop(['Genres','Title'],axis =1)
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,Children's,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12.0,1193,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15.0,1193,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,17.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
data.columns

Index(['UserID', 'MovieID', 'Rating', 'Drama', 'Animation', 'Musical',
       'Action', 'Comedy', 'Adventure', 'Romance', 'Children's', 'Thriller',
       'Crime', 'Western', 'Documentary', 'Mystery', 'Horror', 'Sci-Fi',
       'Film-Noir', 'War', 'Fantasy'],
      dtype='object')

In [71]:
# Because of merging some null values are created
data.isnull().sum()

UserID         282
MovieID          0
Rating         282
Drama            0
Animation        0
Musical          0
Action           0
Comedy           0
Adventure        0
Romance          0
Children's       0
Thriller         0
Crime            0
Western          0
Documentary      0
Mystery          0
Horror           0
Sci-Fi           0
Film-Noir        0
War              0
Fantasy          0
dtype: int64

In [72]:
#WE simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [73]:
data.isnull().sum()

UserID         0
MovieID        0
Rating         0
Drama          0
Animation      0
Musical        0
Action         0
Comedy         0
Adventure      0
Romance        0
Children's     0
Thriller       0
Crime          0
Western        0
Documentary    0
Mystery        0
Horror         0
Sci-Fi         0
Film-Noir      0
War            0
Fantasy        0
dtype: int64

In [74]:
#By different meathods I found 8 cluster are better 
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans()

In [75]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

152873    6
208385    5
108253    7
342108    6
237508    3
197965    3
161032    6
311551    1
154105    0
211259    5
Name: Cluster, dtype: int32

In [76]:
data['Cluster'].value_counts()

3    58010
1    55504
5    54907
6    51314
7    48127
2    45395
4    40971
0    39936
Name: Cluster, dtype: int64

In [77]:
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,Children's,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy,Cluster
0,1.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
1,2.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2,12.0,1193,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
3,15.0,1193,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
4,17.0,1193,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5


In [78]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie 
# the Cluster which occurs maximum number of times is alloted to the movie  
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['MovieID'][d[0]],int(b.idxmax())]
    e.append(c)

In [79]:
data.groupby("MovieID").apply(lambda x: fi(x))

""


In [80]:
e = pd.DataFrame(e)

In [81]:
e.head()

,0,1
0,1,0
1,2,4
2,3,4
3,4,0
4,5,0


In [82]:
# I Dont know why always the column name shift according to its will :(
# Here just the column names are swapped
e.rename(columns = {0:'MovieID',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)

In [83]:
e.head(10)

,MovieID,Cluster
0,1,0
1,2,4
2,3,4
3,4,0
4,5,0
5,6,4
6,7,0
7,8,4
8,9,4
9,10,0


In [84]:
data1 = pd.read_csv('movies data.csv')
new_data = pd.merge(e , data1 , how='outer', on='MovieID')

In [85]:
# These were the movies we deleted while merging the file  
new_data.isnull().sum()

MovieID         0
Cluster       282
Unnamed: 0      0
Title           0
Genres          0
dtype: int64

In [86]:
# We can delete the movies but I just label them randomly :)
new_data.fillna(random.randint(0,8),inplace=True)

In [87]:
new_data.isnull().sum()

MovieID       0
Cluster       0
Unnamed: 0    0
Title         0
Genres        0
dtype: int64

In [88]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['MovieID'][i] , new_data['Title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.MovieID == l]

In [89]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [92]:
main()

Select The Movies Id you would like to watch:
1665--->Bean (1997)
2183--->Man Who Knew Too Much, The (1956)
57--->Home for the Holidays (1995)
3320--->Mifune (Mifunes sidste sang) (1999)
3278--->Gendernauts (1999)
3848--->Silent Fall (1994)
2864--->Splendor (1999)
824--->Kaspar Hauser (1993)
1862--->Species II (1998)
1673--->Boogie Nights (1997)
3414--->Love Is a Many-Splendored Thing (1955)
3427--->Coogan's Bluff (1968)
2650--->Ghost of Frankenstein, The (1942)
1161--->Tin Drum, The (Blechtrommel, Die) (1979)
44--->Mortal Kombat (1995)
--------------------------------------------------------------------
1
643                     Country Life (1994)
89                       Mary Reilly (1996)
385                        8 Seconds (1994)
497                Romeo Is Bleeding (1993)
45     How to Make an American Quilt (1995)
31                    Twelve Monkeys (1995)
494           Road to Wellville, The (1994)
288          Quick and the Dead, The (1995)
430                        Firm, T